In [57]:
# importing libraries
import numpy as np
import math
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import seaborn as sns
try:
    from transformers import AutoTokenizer,AutoModelForQuestionAnswering
except:
    %pip install transformers
    from transformers import AutoTokenizer,AutoModelForQuestionAnswering
import typing as ty

from transformers import AutoModel

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# setting sdevice to gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

In [60]:
# using a fine-tuned tokenizer

path = 'csarron/bert-base-uncased-squad-v1' # use this path instead of the drive one

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/dl_project/Epoch_3")
# using a fine-tuned bert model
# using a fine-tuned tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("/content/drive/MyDrive/dl_project/Epoch_3")
# sending model to device (gpu if available)
model.to(device)
# switching from training mode to testing
model.eval()
model.zero_grad()

In [61]:
def predict(inputs : torch.tensor, token_type_ids : torch.tensor = None, \
            position_ids : torch.tensor = None, attention_mask : torch.tensor = None, output_hidden_states = False)\
            -> ty.Tuple[torch.tensor, torch.tensor]:
    # if(inputs is not None):
    #     inputs = inputs.to(torch.long)
    # if(token_type_ids is not None):
    #     token_type_ids = token_type_ids.to(torch.long)
    # if(position_ids is not None):
    #     position_ids = position_ids = position_ids.to(torch.long)
    # if(attention_mask is not None):
    #     attention_mask = attention_mask - attention_mask.to(torch.long)
    output = model(inputs, token_type_ids=token_type_ids,position_ids=position_ids, attention_mask=attention_mask,output_hidden_states = output_hidden_states)
    return output.start_logits, output.end_logits

In [62]:
sep = tokenizer.sep_token_id 
cls = tokenizer.cls_token_id

In [63]:
# def bert_forward_func(inputs)

def construct_inputs(question : str, context : str, cls : torch.tensor, sep : torch.tensor) -> ty.Tuple[torch.tensor, int]:
    """
    Returns
    -------
    input_ids

    input_ids are the sequence of tokens generated after appending 
    [cls] + question + [sep] + context + [sep]

    Here, [sep] is the separator token which is used to mark the end of text,
    while [cls] token is used to begin the sequence of tokens.
    """
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(context, add_special_tokens=False)
    input_ids = [cls] + question_ids + [sep] + text_ids + [sep]
    input_ids = torch.tensor([input_ids],device=device)
    sep_ind = len(question_ids)
    return input_ids, sep_ind

def construct_attention_mask(input_ids : torch.tensor) -> torch.tensor:
    """
    Returns
    -------
    attention mask

    Attention mask is a tensor with all ones representing where to pay attention.
    We are not zero padding input ids, so attention mask with all ones works in
    our case
    """
    attention_mask = torch.ones_like(input_ids)
    return attention_mask

def construct_token_type_ids(input_ids : torch.tensor, sep_ind : int = 0):
    """
    Returns
    -------
    token type ids

    token type ids represent the class of the token, here it is 0 for the tokens
    belonging to question, while 1 for separating token and context tokens
    """
    len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(len)]], device=device)
    return token_type_ids


def construct_position_ids(input_ids : torch.tensor) -> torch.tensor:
    """
    Returns
    -------
    position ids

    position ids are the index(position) of each input id token
    """
    len = input_ids.size(1) 
    position_ids = torch.arange(len, dtype=torch.long, device=device)
    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids

def decipher_output_token(input_ids : torch.tensor, start: torch.tensor, end : torch.tensor) -> str:
    """
    Returns
    -------
    The answer to the question from text
    """
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    start_token = torch.argmax(start)
    end_token = torch.argmax(end)
    output = ' '.join(all_tokens[start_token:end_token+1])
    return output

In [77]:
import textwrap

Context: str = ('The Panthers finished the regular season with a 15–1 record,'
                'and quarterback Cam Newton was named the NFL Most Valuable'
                'Player (MVP). They defeated the Arizona Cardinals 49–15 in the'
                'NFC Championship Game and advanced to their second Super Bowl'
                'appearance since the franchise was founded in 1995. The'
                'Broncos finished the regular season with a 12–4 record, and'
                'denied the New England Patriots a chance to defend their title'
                'from Super Bowl XLIX by defeating them 20–18 in the AFC'
                'Championship Game. They joined the Patriots, Dallas Cowboys,'
                'and Pittsburgh Steelers as one of four teams that have made'
                'eight appearances in the Super Bowl. The 2020 regular season'
                'win/loss ratio for the Michigan Vikings was 656.')

Question: str = 'How many appearances have the Broncos made in the super bowl?'

ground_truth = 'eight'

print(f'Question: {Question}')
print(f'Context: {textwrap.fill(Context, 79)}')

Question: How many appearances have the Broncos made in the super bowl?
Context: The Panthers finished the regular season with a 15–1 record,and quarterback Cam
Newton was named the NFL Most ValuablePlayer (MVP). They defeated the Arizona
Cardinals 49–15 in theNFC Championship Game and advanced to their second Super
Bowlappearance since the franchise was founded in 1995. TheBroncos finished the
regular season with a 12–4 record, anddenied the New England Patriots a chance
to defend their titlefrom Super Bowl XLIX by defeating them 20–18 in the
AFCChampionship Game. They joined the Patriots, Dallas Cowboys,and Pittsburgh
Steelers as one of four teams that have madeeight appearances in the Super
Bowl. The 2020 regular seasonwin/loss ratio for the Michigan Vikings was 656.


In [78]:
Question: str = 'What would cause recession in Europe?'
Context: str = 'The cut in supply of gas from Russia might cause Recession in Europe'

In [79]:
Context = 'Alfred is a white cat and likes apples. Bobby is a black dog which loves mangoes.'
Question = 'What does Alfred like?'

Context = "The TA liked the mid-term report. Hence, he awarded full marks to the team. All the team members were very happy."
Question = 'Why did the team get full marks ?'

Context = 'Following Russian invasion of Ukraine, more than a dozen countries have sanctioned Russia. One of them is USA'
Question = 'Why did countries sanction Russia?'
ground_truth = 'russian invasion of ukraine'

In [80]:
input_ids, sep_ind = construct_inputs(Question,Context,cls,sep)
indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)
attention_mask = construct_attention_mask(input_ids)
token_type_ids = construct_token_type_ids(input_ids,sep_ind)
position_ids = construct_position_ids(input_ids)
start, end = predict(input_ids, token_type_ids, position_ids, attention_mask)

output = model(input_ids, token_type_ids=token_type_ids,
              position_ids = position_ids,
              attention_mask=attention_mask,
              output_hidden_states = True)

print('predicted answer : ', decipher_output_token(input_ids, start, end))

predicted answer :  four
